<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
#!pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [2]:
import random
import numpy as np
import torch

seed = 2022

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
import pandas as pd

df_args = pd.read_csv('../../../Data/arguments-training.tsv',sep = '\t')
df_lbls = pd.read_table('../../../Data/labels-training.tsv')
df = df_args.merge(df_lbls, how="left", on="Argument ID")

# print(df)

from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(df, split="train")
dataset = DatasetDict({ "train": train_dataset, "validation": train_dataset, "test": train_dataset })

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity', '__index_level_0__'],
        num_rows: 5220
    })
    validation: Dataset({
        features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism:

Let's check the first example of the training split:

In [5]:
example = dataset['train'][0]
example

{'Argument ID': 'A01001',
 'Conclusion': 'Entrapment should be legalized',
 'Stance': 'in favor of',
 'Premise': "if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?",
 'Self-direction: thought': 0,
 'Self-direction: action': 0,
 'Stimulation': 0,
 'Hedonism': 0,
 'Achievement': 0,
 'Power: dominance': 0,
 'Power: resources': 0,
 'Face': 0,
 'Security: personal': 0,
 'Security: societal': 1,
 'Tradition': 0,
 'Conformity: rules': 0,
 'Conformity: interpersonal': 0,
 'Humility': 0,
 'Benevolence: caring': 0,
 'Benevolence: dependability': 0,
 'Universalism: concern': 0,
 'Universalism: nature': 0,
 'Universalism: tolerance': 0,
 'Universalism: objectivity': 0,
 '__index_level_0__': 0}

The dataset consists of arguments, labeled with one or more values. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [6]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Argument ID', 'Conclusion', 'Stance', 'Premise', '__index_level_0__']]
labels.sort()
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Achievement',
 'Benevolence: caring',
 'Benevolence: dependability',
 'Conformity: interpersonal',
 'Conformity: rules',
 'Face',
 'Hedonism',
 'Humility',
 'Power: dominance',
 'Power: resources',
 'Security: personal',
 'Security: societal',
 'Self-direction: action',
 'Self-direction: thought',
 'Stimulation',
 'Tradition',
 'Universalism: concern',
 'Universalism: nature',
 'Universalism: objectivity',
 'Universalism: tolerance']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    premise    = examples["Premise"]
    conclusion = examples["Conclusion"]
    # encode them
    encoding = tokenizer(premise, conclusion, padding="max_length", truncation=True, max_length=128)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(premise), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [8]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [9]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [10]:
tokenizer.decode(example['input_ids'])

"[CLS] if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal? [SEP] entrapment should be legalized [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [11]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [12]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Security: societal']

# N-fold cross validation
Prepare the folds, for n-fold cross validation.
We want to use stratification, although we have a multi-label classification problem, which StratifiedKFold cannot handle. (You get an error: "ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.")

A solution is to convert from multi-label to multi-class. Assuming you have n classes and the target variable is a combination of these n classes. We will have (2^n) - 1 combinations (Not including all 0s).

Lets create a new target variable considering each combination as a new label.

In [13]:
n_folds = 1

In [14]:
from sklearn.preprocessing import LabelEncoder

def multilabel_to_string(y):
    return '-'.join(str(int(l.item())) for l in y)

def multilabel_to_multiclass(y):
    y_new = LabelEncoder().fit_transform([multilabel_to_string(l) for l in y])
    return y_new

# multilabel_to_multiclass(encoded_dataset["train"]["labels"])

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [15]:
encoded_dataset.set_format("torch")

In [16]:
from sklearn.model_selection import StratifiedKFold, train_test_split

if n_folds < 2:
    # stratify = multilabel_to_multiclass(encoded_dataset["train"]["labels"])
    X_train, X_test, y_train, y_test = train_test_split(
        np.arange(dataset["train"].num_rows), np.zeros(dataset["train"].num_rows), test_size=0.1, random_state=seed)
    splits = [X_train, X_test]
else:
    # First make the kfold object
    folds = StratifiedKFold(n_splits=n_folds)

    # Now make our splits based off of the labels. 
    # We can use `np.zeros()` here since it only works off of indices, we really care about the labels
    splits = folds.split(np.zeros(dataset["train"].num_rows), multilabel_to_multiclass(encoded_dataset["train"]["labels"]))

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [17]:
from transformers import AutoModelForSequenceClassification

def instantiate_model():
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                               problem_type="multi_label_classification",
                                                               output_hidden_states=False,
                                                               num_labels=len(labels),
                                                               id2label=id2label,
                                                               label2id=label2id)
    return model

model = instantiate_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [18]:
batch_size = 8
metric_name = "f1"
num_train_epochs = 2

In [19]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import multilabel_confusion_matrix
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, true_labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = true_labels
    precision_micro_average = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_micro_average = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    mcm = multilabel_confusion_matrix(y_true=y_true, y_pred=y_pred, labels=np.arange(len(labels)))
    # return as dictionary
    metrics = {'p': precision_micro_average,
               'r': recall_micro_average,
               'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'mcm': mcm.tolist()
              }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        true_labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [21]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [22]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2065,  4372,  6494, 24073,  2064,  3710,  2000,  2062,  4089,
         5425,  2359, 12290,  1010,  2059,  2339,  5807,  1005,  1056,  2009,
         2022,  3423,  1029,   102,  4372,  6494, 24073,  2323,  2022,  3423,
         3550,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [23]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7347, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0848, -0.4281,  0.5540,  0.1601, -0.0590,  0.0067, -0.0180, -0.4107,
         -0.4852,  0.0186, -0.2489, -0.7731,  0.0675,  0.1176,  0.2130,  0.0916,
         -0.0998,  0.2706,  0.8651, -0.2715]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [24]:
fold = 1

metrics = {
    'p': [],
    'r': [],
    'f1': [],
    'roc_auc': [],
    'accuracy': []
}

if n_folds < 2:
    train_idxs, val_idxs = splits
    
    fold_dataset = DatasetDict({
        "train":encoded_dataset["train"].select(train_idxs),
        "validation":encoded_dataset["train"].select(val_idxs),
        "test":encoded_dataset["validation"]
    })
    
    trainer = Trainer(
        model,
        args,
        train_dataset=fold_dataset["train"],
        eval_dataset=fold_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    results = trainer.evaluate()
    for key in metrics:
        metrics[key].append(results.get('eval_' + key))
else:
    for train_idxs, val_idxs in splits:
        print(f"Fold: {fold}")
        ## Important: Don't forget to reload the original model in each fold!
        model = instantiate_model()
        
        fold_dataset = DatasetDict({
            "train":encoded_dataset["train"].select(train_idxs),
            "validation":encoded_dataset["train"].select(val_idxs),
            "test":encoded_dataset["validation"]
        })
        
        trainer = Trainer(
            model,
            args,
            train_dataset=fold_dataset["train"],
            eval_dataset=fold_dataset["validation"],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )
        
        trainer.train()
        results = trainer.evaluate()
        for key in metrics:
            metrics[key].append(results.get('eval_' + key))
        fold += 1
        # print(metrics)

/home/petasis/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4698
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1176
  Number of trainable parameters = 109497620
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,P,R,F1,Roc Auc,Accuracy,Mcm
1,0.411500,0.354770,0.717666,0.258376,0.379958,0.618876,0.042146,"[[[356, 31], [58, 77]], [[338, 16], [134, 34]], [[438, 0], [84, 0]], [[501, 0], [21, 0]], [[398, 2], [120, 2]], [[497, 0], [25, 0]], [[505, 0], [17, 0]], [[476, 0], [46, 0]], [[478, 0], [44, 0]], [[467, 0], [54, 1]], [[280, 44], [87, 111]], [[318, 39], [58, 107]], [[403, 0], [104, 15]], [[436, 1], [74, 11]], [[492, 0], [30, 0]], [[464, 0], [57, 1]], [[286, 46], [94, 96]], [[477, 0], [45, 0]], [[431, 0], [91, 0]], [[459, 0], [63, 0]]]"
2,0.351200,0.338434,0.738197,0.293015,0.419512,0.635965,0.061303,"[[[366, 21], [67, 68]], [[347, 7], [141, 27]], [[438, 0], [84, 0]], [[501, 0], [21, 0]], [[392, 8], [111, 11]], [[497, 0], [25, 0]], [[505, 0], [17, 0]], [[476, 0], [46, 0]], [[478, 0], [44, 0]], [[467, 0], [52, 3]], [[278, 46], [85, 113]], [[324, 33], [67, 98]], [[397, 6], [85, 34]], [[427, 10], [55, 30]], [[492, 0], [30, 0]], [[461, 3], [54, 4]], [[283, 49], [82, 108]], [[477, 0], [26, 19]], [[431, 0], [91, 0]], [[459, 0], [62, 1]]]"


***** Running Evaluation *****
  Num examples = 522
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-588
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-588/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-588/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-588/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-588/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 522
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-1176
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-1176/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-1176/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-1176/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoin

## Results

In [25]:
import pandas as pd
import numpy as np

decimal_digits = 4

data = [[round(np.mean(metrics['p']), decimal_digits),
         round(np.mean(metrics['r']), decimal_digits),
         round(np.mean(metrics['f1']), decimal_digits),
         round(np.mean(metrics['roc_auc']), decimal_digits),
         round(np.mean(metrics['accuracy']), decimal_digits),
        ],
        [round(np.std(metrics['p']), decimal_digits),
         round(np.std(metrics['r']), decimal_digits),
         round(np.std(metrics['f1']), decimal_digits),
         round(np.std(metrics['roc_auc']), decimal_digits),
         round(np.std(metrics['accuracy']), decimal_digits),
        ]]

df = pd.DataFrame(data, index=["", "±"], columns=["P", "R", "F1", "Roc Auc", "Accuracy"])
df
# df.style.hide_index()

,P,R,F1,Roc Auc,Accuracy
,0.7382,0.293,0.4195,0.636,0.0613
±,0.0000,0.000,0.0000,0.000,0.0000


## Evaluate

After training, we evaluate our model on the validation set.

In [33]:
results = trainer.evaluate()
results

***** Running Evaluation *****
  Num examples = 522
  Batch size = 8


{'eval_loss': 0.33843421936035156,
 'eval_p': 0.7381974248927039,
 'eval_r': 0.293015332197615,
 'eval_f1': 0.4195121951219512,
 'eval_roc_auc': 0.635964976848894,
 'eval_accuracy': 0.06130268199233716,
 'eval_mcm': [[[366, 21], [67, 68]],
  [[347, 7], [141, 27]],
  [[438, 0], [84, 0]],
  [[501, 0], [21, 0]],
  [[392, 8], [111, 11]],
  [[497, 0], [25, 0]],
  [[505, 0], [17, 0]],
  [[476, 0], [46, 0]],
  [[478, 0], [44, 0]],
  [[467, 0], [52, 3]],
  [[278, 46], [85, 113]],
  [[324, 33], [67, 98]],
  [[397, 6], [85, 34]],
  [[427, 10], [55, 30]],
  [[492, 0], [30, 0]],
  [[461, 3], [54, 4]],
  [[283, 49], [82, 108]],
  [[477, 0], [26, 19]],
  [[431, 0], [91, 0]],
  [[459, 0], [62, 1]]],
 'eval_runtime': 7.234,
 'eval_samples_per_second': 72.159,
 'eval_steps_per_second': 9.124,
 'epoch': 2.0}

## Inference

Let's test the model on a new sentence:

In [27]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [28]:
logits = outputs.logits
logits.shape

torch.Size([1, 20])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [29]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Achievement']


In [30]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
hidden_states = outputs.hidden_states
hidden_states